In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
def pos(x):
    y = x.copy()
    y[x > 0] = x[x > 0]
    y[x <= 0] = 0
    return y

In [ ]:
def final_basis_normal(i, x, h, x_0):
    a = pos((x - x_0) / h - (i - 2)) ** 3
    b = 4 * pos((x - x_0) / h - (i - 1)) ** 3
    c = 6 * pos((x - x_0) / h - (i - 0)) ** 3
    d = 4 * pos((x - x_0) / h - (i + 1)) ** 3
    e = pos((x - x_0) / h - (i + 2)) ** 3

    f = 0.25 * (a - b + c - d + e)

    return f

# Biharmonic equation solution

In [ ]:
x0 = 0
xN = 1

y0 = 0
yN = 1

Nx = 11
Ny = 11

X, h_1 = np.linspace(x0, xN, Nx, retstep=True)
Y, h_2 = np.linspace(y0, yN, Ny, retstep=True)

In [ ]:
x = np.linspace(x0 - 2 * h_1, xN + 2 * h_1, 1000)
y = np.linspace(y0 - 2 * h_2, yN + 2 * h_2, 1000)
IX, IY = np.meshgrid(X, Y, indexing="xy")

n_points_x = X.shape[0]
n_points_y = Y.shape[0]

# Computed integrals

In [ ]:
x_axis_xx = np.array(
    [
        3 / 8 / h_1**4,
        0,
        -27 / 8 / h_1**4,
        6 / h_1**4,
        -27 / 8 / h_1**4,
        0,
        3 / 8 / h_1**4,
    ]
)
y_axis_xx = np.array(
    [1 / 2240, 3 / 56, 1991 / 2240, 151 / 140, 1991 / 2240, 3 / 56, 1 / 2240]
)

In [ ]:
x_axis_yy = np.array(
    [1 / 2240, 3 / 56, 1991 / 2240, 151 / 140, 1991 / 2240, 3 / 56, 1 / 2240]
)
y_axis_yy = np.array(
    [
        3 / 8 / h_2**4,
        0,
        -27 / 8 / h_2**4,
        6 / h_2**4,
        -27 / 8 / h_2**4,
        0,
        3 / 8 / h_2**4,
    ]
)

In [ ]:
x_axis_xy = np.array(
    [
        -3 / 160 / h_1**2,
        -9 / 20 / h_1**2,
        -9 / 32 / h_1**2,
        3 / 2 / h_1**2,
        -9 / 32 / h_1**2,
        -9 / 20 / h_1**2,
        -3 / 160 / h_1**2,
    ]
)
y_axis_xy = np.array(
    [
        -3 / 160 / h_2**2,
        -9 / 20 / h_2**2,
        -9 / 32 / h_2**2,
        3 / 2 / h_2**2,
        -9 / 32 / h_2**2,
        -9 / 20 / h_2**2,
        -3 / 160 / h_2**2,
    ]
)

In [ ]:
xx = np.outer(x_axis_xx, y_axis_xx)
xy = np.outer(x_axis_xy, y_axis_xy)
yy = np.outer(x_axis_yy, y_axis_yy)
right = xx + 2 * xy + yy

In [ ]:
matrix = np.zeros((n_points_y * n_points_x, n_points_y * n_points_x))
b = np.zeros(n_points_y * n_points_x)

f = np.ones_like(IX)

for i in range(n_points_y * n_points_x):
    idx = np.unravel_index(i, (n_points_y, n_points_x))
    tmp = np.zeros((n_points_y, n_points_x))

    i_sl_b, i_sl_e = max(0, idx[0] - 3), min(idx[0] + 4, n_points_y)
    j_sl_b, j_sl_e = max(0, idx[1] - 3), min(idx[1] + 4, n_points_x)

    if (idx[0] >= 3) and (idx[0] <= n_points_y - 4):
        ri_sl_b = 0
    elif idx[0] < 3:
        ri_sl_b = 3 - idx[0]
    else:
        ri_sl_b = 0

    if (idx[0] >= 3) and (idx[0] <= n_points_y - 4):
        ri_sl_e = 7
    elif idx[0] < 3:
        ri_sl_e = 7
    else:
        ri_sl_e = n_points_y - idx[0] + 3

    if (idx[1] >= 3) and (idx[1] <= n_points_x - 4):
        rj_sl_b = 0
    elif idx[1] < 3:
        rj_sl_b = 3 - idx[1]
    else:
        rj_sl_b = 0

    if (idx[1] >= 3) and (idx[1] <= n_points_x - 4):
        rj_sl_e = 7
    elif idx[1] < 3:
        rj_sl_e = 7
    else:
        rj_sl_e = n_points_x - idx[1] + 3

    tmp[i_sl_b:i_sl_e, j_sl_b:j_sl_e] = (
        2 * right[ri_sl_b:ri_sl_e, rj_sl_b:rj_sl_e] * h_1 * h_2
    )

    matrix[i] = tmp.flatten()

    b[i] = 2 * 3 / 2 * h_1 * 3 / 2 * h_2 * f[idx]

In [ ]:
C = np.linalg.solve(matrix, b).reshape(n_points_y, n_points_x)

In [ ]:
xx, yy = np.meshgrid(x, y, indexing="ij")
zz = np.zeros_like(xx)

In [ ]:
for i in tqdm(range(IY.shape[0])):
    for j in range(IX.shape[1]):
        y_part = final_basis_normal(i, y, h_2, y0)
        x_part = final_basis_normal(j, x, h_1, x0)
        zz = zz + C[i, j] * np.outer(x_part, y_part)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(6, 6))
ax = plt.axes(projection="3d")
ax.plot_surface(xx, yy, zz, cmap="magma")
ax.tick_params(axis="both", which="major", labelsize=7)
ax.set_title(f"$h_{1}=h_{2}={h_1}$")
fig.tight_layout()
plt.show()